
# Question 1 — Nettoyer *Sherlock Holmes* (expressions régulières)

Dans ce travail, vous allez préparer des textes du **Projet Gutenberg**. Les fichiers contiennent souvent un **en‑tête**, une **table des matières** et une **licence**. Votre objectif est de concevoir des **expressions régulières** pour retirer ces sections de manière reproductible.

Le travail est découpé en **quatre sections** :

1. **Téléchargement d’un livre** (*The Sign of the Four*).
2. **Retrait de la licence** sur ce livre en utilisant une expression régulière et en fournissant votre version.
3. **Conception et application de vos propres expressions régulières** (retraits de l’en‑tête, de la table des matières) **sur ce livre**.
4. **Validation** des mêmes expressions régulières **sur un autre livre** (*The Adventures of Sherlock Holmes*).

> **Règle importante :** vous devez utiliser des **expressions régulières**. Retirer des lignes par simple indexation/slicing d’un tableau **n’est pas accepté**.



## Section 1 — Téléchargement d'un livre (*The Sign of the Four*)

Nous commençons par définir une fonction qui **télécharge** les livres numériques du **Projet Gutenberg**. Nous fournissons aussi une fonction pour afficher le texte capturé par les expressions régulières ainsi qu'une fonction qui permet d'**afficher** une partie de ces documents.

In [38]:
import re
import requests
from typing import Optional, Pattern, Match

def download_ebook(title: str, url: str) -> str:
    """Télécharge un livre texte brut depuis Gutenberg et retourne le contenu (UTF‑8)."""
    resp = requests.get(url, timeout=30)
    resp.raise_for_status()
    resp.encoding = "utf-8"
    text = resp.text
    print(f"Téléchargé: {title} — {len(text.splitlines())} lignes")
    return text

def display_text(text: str, nb_lines: int = 20, from_start: bool = True):
    """Affiche un nombre de lignes depuis le début ou la fin."""
    lines = text.splitlines()
    if from_start:
        print(f"— First {nb_lines} lines —")
        print("\n".join(lines[:nb_lines]))
    else:
        print(f"\n— Last {nb_lines} lines —")
        print("\n".join(lines[-nb_lines:]))

def display_regex_block(text: str, pattern: Pattern[str], label: str):
    """Trouve et affiche **tout le bloc** correspondant à un motif (regex) avec le nombre de lignes exact.
    """
    m: Optional[Match[str]] = pattern.search(text)
    if not m:
        print(f"[Avertissement] Bloc '{label}' non détecté avec le motif proposé.")
        return
    block = m.group(0)
    block_lines = block.splitlines()
    print(f"\n===== {label} (longueur: {len(block_lines)} lignes) =====\n")
    print(block)


In [39]:
# Télécharger le livre. À tout moment, vous pouvez exécuter cette cellule pour recharger le livre.
first_book_title = "The Sign of the Four"
first_book_url = "https://www.gutenberg.org/files/2097/2097-0.txt"
first_book = download_ebook(first_book_title, first_book_url)

Téléchargé: The Sign of the Four — 4559 lignes


## Section 2 — Retrait de la licence (sur *The Sign of the Four*)

Dans cette section, nous appliquons une **expression régulière proposée par un agent conversationnel en ligne** afin **retirer** la **licence** à la fin du livre. Cette regex sert de point de départ, car **vous aurez l’occasion d'améliorer cette expression régulière et vous devrez justifier vos choix en comparant les deux expressions régulières.**

Afin de vous aider voici une description simple de la licence :
- Se trouve à **la fin** du fichier.
- Commence avec la ligne : `*** END OF THE PROJECT GUTENBERG EBOOK ...`.
- Doit être **retirée jusqu’à la fin** du fichier.
- La section peut être **très longue** (souvent **300+ lignes**) ou **très courte** comme dans le premier livre.

Nous allons commencer par afficher quelques lignes à la fin du fichier pour donner une idée du texte de la licence. Gardez en tête que la licence complète est beaucoup plus longue.

In [40]:
display_text(first_book, 10, from_start=False)


— Last 10 lines —

“The division seems rather unfair,” I remarked. “You have done all the
work in this business. I get a wife out of it, Jones gets the credit,
pray what remains for you?”

“For me,” said Sherlock Holmes, “there still remains the
cocaine-bottle.” And he stretched his long white hand up for it.


*** END OF THE PROJECT GUTENBERG EBOOK 2097 ***


In [41]:
# 1) Création de l'expression régulière.
regex_license_llm = r"(?im)(?:^\s*\*{3}\s*END OF (?:THE|THIS)\s+PROJECT GUTENBERG E[- ]?BOOK\b.*|^\s*\*{3}\s*START:\s*FULL LICENSE\s*\*{3}\b.*|^\s*End of (?:the )?Project Gutenberg(?:\'s)?\b.*)[\s\S]*\Z"
license_llm_pattern = re.compile(regex_license_llm, flags=0)

# 2) Afficher le texte capturé par l'expression régulière.
display_regex_block(first_book, license_llm_pattern, "Bloc LICENCE")

# 3) Appliquer la suppression
first_book_no_license_llm = license_llm_pattern.sub("", first_book)


===== Bloc LICENCE (longueur: 3 lignes) =====



*** END OF THE PROJECT GUTENBERG EBOOK 2097 ***



In [42]:
# Affichage après le retrait de la licence. Constatez la différence.
display_text(first_book_no_license_llm, 10, from_start=False)


— Last 10 lines —
had, as I surmised, a confederate in the house, who could be none other
than Lal Rao, the butler: so Jones actually has the undivided honour of
having caught one fish in his great haul.”

“The division seems rather unfair,” I remarked. “You have done all the
work in this business. I get a wife out of it, Jones gets the credit,
pray what remains for you?”

“For me,” said Sherlock Holmes, “there still remains the
cocaine-bottle.” And he stretched his long white hand up for it.


C'est à votre tour de créer et justifier votre expression régulière pour retirer la licence à la fin du livre.

In [43]:
# 1) TODO — ÉCRIVEZ VOTRE REGEX LICENCE
regex_license = (
    r"(?mi)"                             # m: ^ au début de ligne ; i: insensible à la casse
    r"^\s*\*{3,}\s*END\s+OF\s+(?:THE|THIS)\s+PROJECT\s+GUTENBERG\s+E(?:[-\s]?BOOK)\b[^\n]*\r?$"      # ligne déclencheuse: *** END OF THE PROJECT GUTENBERG EBOOK
    r"[\s\S]*\Z"                          # puis tout jusqu'à la fin du fichier
)
license_pattern = re.compile(regex_license, flags=0)


# 2) Afficher le texte capturé par l'expression régulière.
display_regex_block(first_book, license_pattern, "Bloc LICENCE")

# 3) Appliquer la suppression
first_book_no_license = license_pattern.sub("", first_book)


===== Bloc LICENCE (longueur: 3 lignes) =====



*** END OF THE PROJECT GUTENBERG EBOOK 2097 ***



In [44]:
# Affichage après le retrait de la licence. Devrait être pareil
display_text(first_book_no_license, 10, from_start=False)


— Last 10 lines —
had, as I surmised, a confederate in the house, who could be none other
than Lal Rao, the butler: so Jones actually has the undivided honour of
having caught one fish in his great haul.”

“The division seems rather unfair,” I remarked. “You have done all the
work in this business. I get a wife out of it, Jones gets the credit,
pray what remains for you?”

“For me,” said Sherlock Holmes, “there still remains the
cocaine-bottle.” And he stretched his long white hand up for it.


### **EXPLICATION DE NOTRE EXPRESSION REGULIERE ET COMPARAISON AVEC L'EXPRESSION INITIALE**

### Pourquoi ce motif ?

**(?mi)**
- **m** : active le mode multi-ligne → ^ et $ correspondent au début/fin de **ligne**.  
- **i** : insensible à la casse.

**^\s*\\*{3,}\s***
- Autorise des espaces facultatifs avant/après les astérisques.  
- \*{3,} = au moins **trois** * (au cas où certaines éditions en ont davantage).  
- On garde \s* pour absorber proprement d’éventuels blancs en début de ligne, évitant de laisser des **blancs résiduels** après suppression.

**END\s+OF\s+(?:THE|THIS)\s+PROJECT\s+GUTENBERG\s+E(?:[-\s]?BOOK)\b**
- Cible la structure requise : “END OF … PROJECT GUTENBERG … EBOOK”.  mais de manière plus générale
- (?:THE|THIS) couvre une petite variation fréquente (la consigne ne le précise pas, on n'est pas certains que la variante avec THIS existe, on la donne par souci de généralisation)
- E(?:[-\s]?BOOK) accepte **EBOOK / E-BOOK / E BOOK**, Idem que THE/THIS, sans multiplier les cas exotiques.

**[^\n]*\r?$**
- Capture le **reste de la ligne**, par exemple. un numéro d’ebook et/ou des *** finaux.  
- Compatible **LF/CRLF** (\r?).

**[\s\S]*\Z**
- Supprime tout ce qui suit jusqu’à la fin du fichier, quelle que soit la longueur de la licence.

**En résumé** : on ne “recopie” pas un texte spécifique (numéro, titre, ponctuation exacte) ; on décrit la forme de la ligne déclencheur + on enlève jusqu’à EOF. Le motif reste simple, lisible, et assez tolérant pour fonctionner sur au moins les deux livres du TP.


### Comparaison avec l’expression initiale (point de départ)

**Couverture**
- L’expression initiale empilait plusieurs variantes (dont certaines non demandées par la consigne).  
- Notre version réduit le périmètre au cas attendu :  
  *** END OF (THE|THIS) PROJECT GUTENBERG EBOOK …,  
  tout en restant légèrement généraliste (espaces/astérisques/séparateurs).

**Simplicité & lisibilité**
- Un **seul ancrage** ^ au début de ligne et une fin de ligne explicite $ avant [\s\S]*\Z.  
- Un **scénario clair** + des tolérances ciblées (THE|THIS, EBOOK/E-BOOK/E BOOK`, nombre d’astérisques, espaces).  
- Plus **facile à expliquer** et à relire que des branches multiples.

**Généralité tout en capturant “juste ce qu’il faut”**
- Notre regex est à priori suffisante pour le (les deux) livres du TP et des cas voisins, sans “sur-généraliser” (pas d’hypothèses grammaticalement douteuses, pas de variantes éloignées).  
- Compatible LF/CRLF et mise en page variée (blancs, ponctuation finale sur la ligne).


Finalement, notre regex est **plus simple**, **ciblée** sur la **structure** demandée, tout en restant **assez générale** pour fonctionner de façon reproductible sur **au moins deux éditions** sans copier-coller un texte spécifique.  
Elle **améliore la clarté** par rapport à l’expression initiale trop permissive, tout en respectant **l’esprit de la consigne**.


## Section 3 — Vos expressions régulières (sur *The Sign of the Four*)

Vous devez écrire des **expressions régulières** (pas de suppression par indexation/liste) pour retirer successivement **l’en‑tête**, **la table des matières** et **la licence**.

Avant de créer vos expressions régulières voici un aperçu du début du livre avec l'entête.

In [45]:
display_text(first_book, 30, from_start=True)

— First 30 lines —
*** START OF THE PROJECT GUTENBERG EBOOK 2097 ***




The Sign of the Four

by Arthur Conan Doyle




Contents


   Chapter I. The Science of Deduction
   Chapter II. The Statement of the Case
   Chapter III. In Quest of a Solution
   Chapter IV. The Story of the Bald-Headed Man
   Chapter V. The Tragedy of Pondicherry Lodge
   Chapter VI. Sherlock Holmes Gives a Demonstration
   Chapter VII. The Episode of the Barrel
   Chapter VIII. The Baker Street Irregulars
   Chapter IX. A Break in the Chain
   Chapter X. The End of the Islander
   Chapter XI. The Great Agra Treasure
   Chapter XII. The Strange Story of Jonathan Small






### 3.1 — Retirer l’**en‑tête**

**Description simple :**
- L’en‑tête se trouve **avant** le contenu réel du livre.
- Se termine par la ligne suivante : *** START OF THE PROJECT GUTENBERG EBOOK ...`
- L'en-tête peut être sur plusieurs lignes.

**Tâches :**
* 1. Écrire une expression régulière **robuste** pour capturer **tout** depuis le **début** jusqu’**après** la ligne “START OF …”.
* 2. **Afficher** exactement le bloc détecté.
* 3. **Retirer** ce bloc du texte.


In [72]:
# 1) TODO — ÉCRIVEZ VOTRE REGEX EN‑TÊTE
header_regex = (
    r"(?mi)"
    r"\A(?:\ufeff)?" # capture des éventuels caractères invisibles
    r"[\s\S]*?"
    r"^\s*\*{3}\s*START\s+OF\s+(?:THE|THIS)\s+PROJECT\s+GUTENBERG\s+E(?:[-\s]?BOOK)[^\r\n]*\r?$"
    r"[ \t\r\n]*"   # consomme les lignes vides qui suivent
)
header_pattern = re.compile(header_regex, flags=0)

# 2) Afficher le texte capturé par l'expression régulière.
display_regex_block(first_book_no_license, header_pattern, "Bloc EN‑TÊTE")

# 3) Appliquer la suppression
first_book_no_header = header_pattern.sub("", first_book_no_license)


===== Bloc EN‑TÊTE (longueur: 5 lignes) =====

*** START OF THE PROJECT GUTENBERG EBOOK 2097 ***







**EXPLICATION DE NOTRE EXPRESSION REGULIERE ET COMPARAISON AVEC L'EXPRESSION DE LA LICENCE**

- **(?mi) :** m active le mode multi-ligne → ^ et \$ fonctionnent par ligne, i rend la recherche insensible à la casse

- **\A :** ancrage au tout début du fichier (différent de ^ qui cible début de ligne).

- **(?:\ufeff)? :** capture facultativement un éventuel BOM (caractère invisible présent dans certains fichiers).

- **[\s\S]*? :** capture tout caractère, y compris les retours à la ligne, en mode non gourmand (s’arrête dès qu’on rencontre la ligne déclencheur).

- ******^\s*\*{3}^\s*\*{3}\s*START\s+OF\s+(?:THE|THIS)\s+PROJECT\s+GUTENBERG\s+E(?:[-\s]?BOOK)[^\r\n]*\r?$ :**
Repère la ligne déclencheur telle qu'indiquée dans l'énoncé, avec la même généralisation que pour la licence.

- **[ \t\r\n]** : capture les lignes vides qui suivent immédiatement la ligne déclencheur. Cela permet d’éviter d’avoir un texte qui commence par des blancs inutiles.


**Comparaison avec la regex de la licence :**  
Pour la licence, on capture depuis la ligne END OF ... jusqu’à la fin du fichier ([\s\S]*\Z).
Pour l’en-tête, on fait l’inverse : on capture depuis le début du fichier (\A) jusqu’à la ligne START OF ... (incluse, plus ses lignes vides).
Ainsi, le bloc retiré correspond à toutes les lignes avant le titre, et incluant *** START OF THE PROJECT GUTENBERG EBOOK ....
Après suppression, le texte démarre directement au titre du livre (The Sign of the Four), sans espaces superflus.

### 3.2 — Retirer la **table des matières**

**Description simple :**
- Commence par la ligne `Contents`/`CONTENTS` (pouvant être précédé d’espaces) et se trouve après le titre et l'auteur qui suivent l'en-tête.
- Une liste d’entrées de chapitres (chiffres romains possibles).
- Se termine par une ligne vide suivant le dernier chapitre.
- Peut contenir des lignes vides.

**Tâches :**
1. Écrire une regex **robuste** qui détecte la section table des matières depuis `Contents` jusqu’à **la première ligne vide**.
2. **Afficher** le bloc détecté.
3. **Retirer** ce bloc du texte obtenu après l’en‑tête.


In [ ]:
# 1) TODO — ÉCRIVEZ VOTRE REGEX TABLE DES MATIÈRES
toc_regex = (
    r"(?mi)"                         # m: ^/$ par ligne | i: casse
    r"^\s*CONTENTS?\s*$"             # 'Contents' / 'CONTENTS'
    r"(?:\r?\n[ \t]*$)*"             # lignes vides sous 'Contents'
    r"(?:"                           # >= 1 entrée
        r"\r?\n[ \t]*(?:"
            r"[A-Za-z][^\r\n]*"        # (A) Titre de chapitre "libre" (ligne non vide) 
            #r"(?:CHAPTER)\s\.?.*"  # Imposer un patron "CHAPTER" permettrait de réduire la confusion engendrée par la consigne "peut contenir des lignes vides"
            r"|"
            r"[IVXLCDM]+\.?\s+.*"      # (B) Chiffres romains + titre
        r")"
        #r"(?:\r?\n[ \t]*$)*"        # lignes vides autorisées entre les chapitres 
                                     # (selon l'interprétation de l'énoncé, mais nécessiterait de décommenter le patron fort "CHAPTER" ci-dessus et de commenter l'alternative r"[A-Za-z][^\r\n]*")
    r")+"
    r"\r?\n[ \t]*\r?\n"             # 1ère ligne vide APRÈS la dernière entrée = borne de fin d'après l'énoncé
)
toc_pattern = re.compile(toc_regex, flags=0)

# 2) Afficher le texte capturé par l'expression régulière.
display_regex_block(first_book_no_header, toc_pattern, "Bloc TABLE DES MATIÈRES")

# 3) Appliquer la suppression
first_book_cleaned = toc_pattern.sub("", first_book_no_header)


===== Bloc TABLE DES MATIÈRES (longueur: 20 lignes) =====





Contents


   Chapter I. The Science of Deduction
   Chapter II. The Statement of the Case
   Chapter III. In Quest of a Solution
   Chapter IV. The Story of the Bald-Headed Man
   Chapter V. The Tragedy of Pondicherry Lodge
   Chapter VI. Sherlock Holmes Gives a Demonstration
   Chapter VII. The Episode of the Barrel
   Chapter VIII. The Baker Street Irregulars
   Chapter IX. A Break in the Chain
   Chapter X. The End of the Islander
   Chapter XI. The Great Agra Treasure
   Chapter XII. The Strange Story of Jonathan Small




**EXPLICATION DE NOTRE REGEX POUR RETIRER LA TABLE DES MATIERES**

Le bloc détecté inclut Contents, les éventuels blancs, toutes les entrées (I… XII…) et se termine à la première ligne vide suivante. Après suppression, le texte commence directement au chapitre I, sans empiéter sur le contenu narratif.

- **(?m)** active le mode multi-ligne ; ^ et $ visent le début/fin de ligne, ce qui permet d’attraper précisément la ligne Contents et chaque entrée du sommaire.

- **(?i)** couvre Contents / CONTENTS (flags combinés (?mi)).

- Les classes \s et [ \t]* absorbent les variations d’espacement (espaces, tabulations, lignes vides optionnelles).

On accepte deux formes d'entrée selon l’édition : 'Entrée de chapitre LIBRE' + ... (par exemple, Chapter + 'titre du chapitre' + chiffres romains (I, II, …)), ou chiffres romains + ... (par exemple,  II. The Red-Headed League).

Le **+** sur le bloc d’entrées impose au moins une entrée (on n’efface pas un simple “Contents” isolé).

La première ligne vide après la dernière entrée (\r?\n[ \t]*\r?\n) stoppe la capture exactement où demandé, sans “manger” le début du Chapitre I. L’utilisation d’une ligne vide comme sentinelle contrôle la gourmandise du motif.

**\r?\n** gère proprement les fins de ligne Windows/Unix.

Ancrer clairement le début (^…Contents…$) et la fin (ligne vide sentinelle) limite faux positifs/négatifs, tout en exigeant une entrée structurée.

**Choix d’interprétation de l’énoncé (“peut contenir des lignes vides”)**

L’énoncé peut prêter à confusion : 
- Dans notre version, une entrée = un titre libre ou un chiffre romain, nous permettons des lignes vides après Contents et utilisons une ligne vide terminale pour clore la section. Nous n’autorisons pas de lignes vides entre deux entrées de chapitres, sinon il serait impossible de différencier une entrée de table des matières et une prose (par exemple si on a une ligne vide entre la dernière entrée de chapitre, le premier paragraphe de la prose, une ligne vide, puis le deuxième paragraphe de la prose alors... on ne s'arrêterait jamais)
- Cette interprétation rend l’extraction plus déterministe et évite de capturer par erreur une ligne narrative isolée.
- Variante (si l’on souhaite autoriser des blancs entre entrées de chapitres) : il suffit d’ajouter (?:\r?\n[ \t]*$)* après chaque entrée, à condition d’utiliser des patrons d’entrée plus “forts” (par exemple "CHAPTER\s+\.?\s+.+" et/ou [IVXLCDM]+\.?\s+.+) pour ne pas confondre prose et sommaire.
- Ce sont des précautions qu'on prend : dans l'exemple du TP il y a de toute façon plus d'une ligne vide après la dernière entrée, donc pas de souci à se poser par rapport à cela.

Dans les cellules suivantes, vous pouvez observer l'effet de retirer toutes les sections sur le document.

In [74]:
print("\n--- Début (après tous retraits) — 20 lignes ---\n")
display_text(first_book_cleaned, 20, from_start=True)


--- Début (après tous retraits) — 20 lignes ---

— First 20 lines —
The Sign of the Four

by Arthur Conan Doyle



Chapter I
The Science of Deduction


Sherlock Holmes took his bottle from the corner of the mantel-piece and
his hypodermic syringe from its neat morocco case. With his long,
white, nervous fingers he adjusted the delicate needle, and rolled back
his left shirt-cuff. For some little time his eyes rested thoughtfully
upon the sinewy forearm and wrist all dotted and scarred with
innumerable puncture-marks. Finally he thrust the sharp point home,
pressed down the tiny piston, and sank back into the velvet-lined
arm-chair with a long sigh of satisfaction.

Three times a day for many months I had witnessed this performance, but


In [75]:
print("\n--- Fin (après tous retraits) — 20 lignes ---\n")
display_text(first_book_cleaned, 20, from_start=False)


--- Fin (après tous retraits) — 20 lignes ---


— Last 20 lines —

“Yes,” he answered, “there are in me the makings of a very fine loafer
and also of a pretty spry sort of fellow. I often think of those lines
of old Goethe,—

Schade dass die Natur nur _einen_ Mensch aus Dir schuf,
Denn zum würdigen Mann war und zum Schelmen der Stoff.


“By the way, _à propos_ of this Norwood business, you see that they
had, as I surmised, a confederate in the house, who could be none other
than Lal Rao, the butler: so Jones actually has the undivided honour of
having caught one fish in his great haul.”

“The division seems rather unfair,” I remarked. “You have done all the
work in this business. I get a wife out of it, Jones gets the credit,
pray what remains for you?”

“For me,” said Sherlock Holmes, “there still remains the
cocaine-bottle.” And he stretched his long white hand up for it.


**CONCLUSION**

On a bien le résultat que l'on souhaitait : Un texte qui commence au titre du livre, sans en-tête ni table de matière ou espaces superflus, et qui finit à la dernière phrase du livre, sans licence ni espaces superflus.


## Section 4 — Validation sur *The Adventures of Sherlock Holmes*

Téléchargez *The Adventures of Sherlock Holmes* et appliquez **les mêmes expressions régulières** pour vérifier leur **généralisation**.

> **Critères** :  
> - Le début ne contient plus d’en‑tête Gutenberg.  
> - La table des matières est supprimée si elle existe dans cette édition.  
> - La licence n’apparaît plus à la fin.  
> - Le texte des chapitres demeure intact.


In [76]:
second_book_title = "The Adventures of Sherlock Holmes"
second_book_url = "https://www.gutenberg.org/cache/epub/1661/pg1661.txt"
second_book = download_ebook(second_book_title, second_book_url)

Téléchargé: The Adventures of Sherlock Holmes — 12306 lignes


In [77]:
display_regex_block(second_book, header_pattern, "Bloc EN‑TÊTE (2e livre)")
second_book_cleaned = header_pattern.sub("", second_book)


===== Bloc EN‑TÊTE (2e livre) (longueur: 27 lignes) =====

﻿The Project Gutenberg eBook of The Adventures of Sherlock Holmes
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Adventures of Sherlock Holmes

Author: Arthur Conan Doyle

Release date: March 1, 1999 [eBook #1661]
                Most recently updated: October 10, 2023

Language: English

Credits: an anonymous Project Gutenberg volunteer and Jose Menendez


*** START OF THE PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES ***







On a bien toutes les lignes qu'il y a avant la phrase déclencheuse *** START OF THE PROJECT GUTENBERG EBOOK ... ***, la phrase en question, ainsi que les blancs superflux qui viennent ensuite.

In [78]:
display_regex_block(second_book, toc_pattern, "Bloc TABLE DES MATIÈRES (2e livre)")
second_book_cleaned = toc_pattern.sub("", second_book_cleaned)


===== Bloc TABLE DES MATIÈRES (2e livre) (longueur: 17 lignes) =====



Contents

   I.     A Scandal in Bohemia
   II.    The Red-Headed League
   III.   A Case of Identity
   IV.    The Boscombe Valley Mystery
   V.     The Five Orange Pips
   VI.    The Man with the Twisted Lip
   VII.   The Adventure of the Blue Carbuncle
   VIII.  The Adventure of the Speckled Band
   IX.    The Adventure of the Engineer’s Thumb
   X.     The Adventure of the Noble Bachelor
   XI.    The Adventure of the Beryl Coronet
   XII.   The Adventure of the Copper Beeches




On a bien toutes les lignes qu'il y a avant la phrase déclencheuse Contents, on capture bien la tables des matières dans le cas où il n'y a que des chiffres romains, ainsi que les blancs superflux qui viennent ensuite.

In [79]:
display_regex_block(second_book, license_pattern, "Bloc LICENCE (2e livre)")
second_book_cleaned = license_pattern.sub("", second_book_cleaned)


===== Bloc LICENCE (2e livre) (longueur: 357 lignes) =====







*** END OF THE PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES ***


    

Updated editions will replace the previous one—the old editions will
be renamed.

Creating the works from print editions not protected by U.S. copyright
law means that no one owns a United States copyright in these works,
so the Foundation (and you!) can copy and distribute it in the United
States without permission and without paying copyright
royalties. Special rules, set forth in the General Terms of Use part
of this license, apply to copying and distributing Project
Gutenberg™ electronic works to protect the PROJECT GUTENBERG™
concept and trademark. Project Gutenberg is a registered trademark,
and may not be used if you charge for an eBook, except by following
the terms of the trademark license, including paying royalties for use
of the Project Gutenberg trademark. If you do not charge anything for
copies of this eBook, complying wi

On a bien tous les blancs superflux avant la licence, la phrase déclencheuse de la licence, ainsi que tout le texte qui vient ensuite.

In [81]:
print("\n--- Début (après tous retraits) — 20 lignes ---\n")
display_text(second_book_cleaned, 20, from_start=True)


--- Début (après tous retraits) — 20 lignes ---

— First 20 lines —
The Adventures of Sherlock Holmes

by Arthur Conan Doyle



I. A SCANDAL IN BOHEMIA


I.

To Sherlock Holmes she is always _the_ woman. I have seldom heard him
mention her under any other name. In his eyes she eclipses and
predominates the whole of her sex. It was not that he felt any emotion
akin to love for Irene Adler. All emotions, and that one particularly,
were abhorrent to his cold, precise but admirably balanced mind. He
was, I take it, the most perfect reasoning and observing machine that
the world has seen, but as a lover he would have placed himself in a
false position. He never spoke of the softer passions, save with a gibe
and a sneer. They were admirable things for the observer—excellent for


In [82]:
print("\n--- Fin (après tous retraits) — 20 lignes ---\n")
display_text(second_book_cleaned, 20, from_start=False)


--- Fin (après tous retraits) — 20 lignes ---


— Last 20 lines —
“You have it, sir, just as it happened.”

“I am sure we owe you an apology, Mrs. Toller,” said Holmes, “for you
have certainly cleared up everything which puzzled us. And here comes
the country surgeon and Mrs. Rucastle, so I think, Watson, that we had
best escort Miss Hunter back to Winchester, as it seems to me that our
_locus standi_ now is rather a questionable one.”

And thus was solved the mystery of the sinister house with the copper
beeches in front of the door. Mr. Rucastle survived, but was always a
broken man, kept alive solely through the care of his devoted wife.
They still live with their old servants, who probably know so much of
Rucastle’s past life that he finds it difficult to part from them. Mr.
Fowler and Miss Rucastle were married, by special license, in
Southampton the day after their flight, and he is now the holder of a
government appointment in the island of Mauritius. As to Miss Violet
Hunter, 

**CONCLUSION**

On a bien ce qu'on voulait pour les deux livres :
- Suppression de l'en-tête
- Suppression de la table des matières
- Suppression de tout le texte de la license
- Suppression des blancs superflux dans les trois parties traitées

Au final, nos regex sont bien généralisées pour les deux livres.